## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.api import widgets
from openbb_terminal.api import openbb
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = widgets.html_report_stylesheet()
with open("./openbb_terminal/reports/OpenBB_reports_logo.png", "rb") as image_file:
    openbb_image_encoded = base64.b64encode(image_file.read())

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "ATOM"
report_name = ""

In [ ]:
author = "Henrique Joaquim"
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
# basic info
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id
ticker = "btc"
cp_id = get_coinpaprika_id(ticker)
basic_info = openbb.crypto.dd.basic(cp_id)
basic_info = basic_info.set_index("Metric")
basic_info

In [ ]:
five_year = datetime.datetime.now() - datetime.timedelta(days=1825)
three_year = datetime.datetime.now() - datetime.timedelta(days=1095)
one_year = datetime.datetime.now() - datetime.timedelta(days=365)
three_months = datetime.datetime.now() - datetime.timedelta(days=90)

dates = {
    "three_months": three_months,
    "one_year": one_year,
    "three_year": three_year,
    "five_year": five_year,
}

In [ ]:
def get_historical_data(ticker, dict_of_dates, is_plot=True):

    list_of_historical_data = []

    for date in dict_of_dates:
        data = openbb.crypto.load(
            symbol=ticker,
            start_date=dict_of_dates[date],
        )

        # we'll be using the adjusted close price as same as close price
        data["Adj Close"] = data["Close"]

        if is_plot:
            fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
            plt.suptitle(
                f"{date} historical data for {ticker.upper()}",
                fontsize=14,
                color="black",
            )
            plt.plot(data.index, data["Close"])
            plt.ylabel("USDT")
            plt.show()

            fig.tight_layout()
            f = io.BytesIO()
            fig.savefig(f, format="svg")
            plot_data = f.getvalue().decode("utf-8")

            list_of_historical_data.append(plot_data)
        else:
            list_of_historical_data.append(data)

    return list_of_historical_data

In [ ]:
historical_data = get_historical_data(ticker, dates)

In [ ]:
historical_data_one_year = get_historical_data(ticker, {"one_year": one_year}, False)[0]
historical_data_one_year

In [ ]:
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import show_quick_performance

show_quick_performance(
    crypto_df=historical_data_one_year,
    symbol=ticker,
    current_currency="USDT",
    source="cctx",
    exchange="binance",
    interval="1440",
)

fig = plt.figure(figsize=(11, 3), dpi=150)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
quick_performance = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.crypto.dd.headlines(ticker, external_axes=[ax], chart=True)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
headlines = f.getvalue().decode("utf-8")

In [ ]:
# fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
# openbb.crypto.chart(
#     historical_data_one_year,
#     symbol=ticker,
#     currency="USDT",
#     source="cctx",
#     exchange="binance",
#     interval="1440",
#     external_axes=[ax1, ax2],
# )
# fig.tight_layout()
# f = io.BytesIO()
# fig.savefig(f, format="svg")
# one_year_candle = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.crypto.chart(
    historical_data_one_year,
    symbol=ticker,
    currency="USDT",
    source="cctx",
    exchange="binance",
    interval="1440",
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
candle_one_year = f.getvalue().decode("utf-8")

In [ ]:
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko

potential_returns = pd.DataFrame()
try:
    current_coin_id = get_coingecko_id(symbol=ticker)
    coin_found = get_coingecko_id(symbol="BTC")

    potential_returns = gecko.get_coin_potential_returns(current_coin_id, coin_found)
    potential_returns = potential_returns.set_index("Coin")

except Exception as e:
    # potential_returns = str("Reached coingecko API limit")
    pass

## Due Diligence

In [ ]:
trading_pair_info = openbb.crypto.dd.trading_pair_info(f"{ticker}-USDT")
trading_pair_info = trading_pair_info.set_index("Metric")
trading_pair_info

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

ath = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    ath = coin.get_all_time_high(currency=cg_id)

except Exception as e:
    # ath = str("Reached coingecko API limit")
    pass

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

atl = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    atl = coin.get_all_time_low(currency=cg_id)

except Exception as e:
    # atl = str("Reached coingecko API limit")
    pass

In [ ]:
# market stats about loaded coin
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id

cp_id = get_coinpaprika_id(ticker)
market = openbb.crypto.dd.mkt(cp_id)
market = market.set_index("exchange")

In [ ]:
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id

# all exchanges where loaded coin is listed
cp_id = get_coinpaprika_id(ticker)
ex = openbb.crypto.dd.ex(cp_id)
ex = ex.set_index("id")
ex

In [ ]:
# last trades
trades = openbb.crypto.dd.trades(symbol=f"{ticker}-USDT")
trades = trades.set_index("time")
trades

In [ ]:
# order book

fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
openbb.crypto.dd.cbbook(symbol=f"{ticker}-USDT", external_axes=[ax], chart=True)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ob = f.getvalue().decode("utf-8")

In [ ]:
# price and supply related metrics for loaded coin
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id

cp_id = get_coinpaprika_id(ticker)
ps = openbb.crypto.dd.ps(cp_id)
ps = ps.set_index("Metric")
ps

In [ ]:
# events
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id

cp_id = get_coinpaprika_id(ticker)
events = openbb.crypto.dd.events(cp_id)
events = events.set_index("date")

In [ ]:
# social media

from openbb_terminal.cryptocurrency.due_diligence.pycoingecko_view import display_social

display_social(ticker)
fig = plt.figure()
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
social = f.getvalue().decode("utf-8")

In [ ]:
# tweets for loaded coin
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coinpaprika_id

cp_id = get_coinpaprika_id(ticker)
tweets = openbb.crypto.dd.twitter(cp_id)
tweets

In [ ]:
# github activity over time

gh = openbb.crypto.dd.gh(ticker)
type(gh)

## TA


In [ ]:
ticker_data = historical_data_one_year.copy()

In [ ]:
# simple moving average
ticker_data.index.names = ["date"]

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.common.ta.ma(
    data=ticker_data["Close"],
    ma_type="SMA",
    symbol=ticker,
    external_axes=[ax],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
sma = f.getvalue().decode("utf-8")

In [ ]:
# exponential moving average
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.common.ta.ma(
    data=ticker_data["Close"],
    ma_type="EMA",
    symbol=ticker,
    external_axes=[ax],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ema = f.getvalue().decode("utf-8")

In [ ]:
# zero lag exponential moving average

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.common.ta.ma(
    data=ticker_data["Close"],
    ma_type="ZLMA",
    symbol=ticker,
    external_axes=[ax],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
zlma = f.getvalue().decode("utf-8")

In [ ]:
# commodity channel index

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.common.ta.cci(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
cci = f.getvalue().decode("utf-8")

In [ ]:
# moving average convergence/divergence

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.common.ta.macd(
    series=ticker_data["Adj Close"],
    symbol=ticker,
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
macd = f.getvalue().decode("utf-8")

In [ ]:
# fisher transform

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(11, 11), dpi=150)
openbb.common.ta.fisher(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2, ax3],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
fisher = f.getvalue().decode("utf-8")

In [ ]:
# aroon indicator

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(11, 11), dpi=150)
openbb.common.ta.aroon(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2, ax3],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
aroon = f.getvalue().decode("utf-8")

In [ ]:
# bollinger bands

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.common.ta.bbands(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bbands = f.getvalue().decode("utf-8")

In [ ]:
# fibonacci retracement

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.common.ta.fib(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
fib = f.getvalue().decode("utf-8")

## QA

In [ ]:
# summary

summary = openbb.common.qa.summary(data=ticker_data)
summary

In [ ]:
# normality statistics and tests

# fig, _ = plt.subplots(figsize=(11, 3), dpi=150)
# openbb.common.qa.normality(
#     data=ticker_data,
#     target="Close",
#     chart=True,
# )
# fig.tight_layout()
# f = io.BytesIO()
# fig.savefig(f, format="svg")

normality = openbb.common.qa.normality(
    data=ticker_data["Close"],
)
normality

In [ ]:
# box and whisker plot

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.common.qa.bw(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    yearly=False,
    external_axes=[ax],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw = f.getvalue().decode("utf-8")

In [ ]:
# rolling mean and std deviation of prices

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.common.qa.rolling(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
rolling = f.getvalue().decode("utf-8")

In [ ]:
# rolling kurtosis of distribution of prices

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.common.qa.kurtosis(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=[ax1, ax2],
    chart=True,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
kurtosis = f.getvalue().decode("utf-8")

## PRED

In [ ]:
# TODO: add prediction

## Render the report template to a file

In [ ]:
body = ""

img = f'<img src="data:image/png;base64,{openbb_image_encoded.decode()}" alt="OpenBB" style="width:144px;">'
body += widgets.header(
    img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>INVESTMENT RESEARCH REPORT:</b> {ticker}",
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Due Diligence",
        "Technical Analysis",
        "Quantitative Analysis",
    ]
)

comment = ""

# Summary tab
htmlcode = ""
htmlcode += widgets.row(
    [widgets.h(3, "Basic information about loaded coin") + basic_info.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Historical data")])
for data in historical_data:
    htmlcode += widgets.row([data])
htmlcode += widgets.row([widgets.h(3, "1-year historical data - in detail")])
htmlcode += widgets.row(
    [widgets.p("Note that the whole analysis is based on this 1-year historical data.")]
)
htmlcode += widgets.row(
    [
        widgets.p("Here is a sample among the 365 days.")
        + historical_data_one_year.sample(n=20).sort_index().to_html()
    ]
)
htmlcode += widgets.row([widgets.h(4, "1-year historical data candlestick chart")])
htmlcode += widgets.row([candle_one_year])
htmlcode += widgets.row([widgets.h(3, "Sentiment analysis")])
htmlcode += widgets.row([headlines])
htmlcode += widgets.row([widgets.h(3, "Potential returns against BTC")])
htmlcode += widgets.row([potential_returns.to_html()])
# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Quick performance")])
# htmlcode += widgets.row([quick_performance])
body += widgets.add_tab("SUMMARY", htmlcode)

# # Due diligence tab
htmlcode = ""
htmlcode += widgets.row(
    [widgets.h(3, f"Trading pair info {ticker}/USDT") + trading_pair_info.to_html()]
)
if not ath.empty:
    htmlcode += widgets.row([widgets.h(3, "All time high")])
    htmlcode += widgets.row([ath.to_html()])
if not atl.empty:
    htmlcode += widgets.row([widgets.h(3, "All time low")])
    htmlcode += widgets.row([atl.to_html()])
htmlcode += widgets.row([widgets.h(3, "Market stats about loaded coin")])
htmlcode += widgets.row([market.sample(n=20).sort_index().to_html()])
htmlcode += widgets.row([widgets.h(3, "Exchanges where loaded coin is listed")])
htmlcode += widgets.row([ex.sample(n=20).sort_index().to_html()])
htmlcode += widgets.row([widgets.h(3, "Last trades")])
htmlcode += widgets.row([trades.tail(20).to_html()])
htmlcode += widgets.row([widgets.h(3, "Order book")])
htmlcode += widgets.row([ob])
htmlcode += widgets.row(
    [widgets.h(3, "Price and supply related metrics for loaded coin")]
)
htmlcode += widgets.row([ps.to_html()])
if not events.empty:
    htmlcode += widgets.row([widgets.h(3, "Events")])
    htmlcode += widgets.row([events.to_html()])
# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Tweets for loaded coin")])
# htmlcode += widgets.row([tweets])
# htmlcode += widgets.row([widgets.h(3, "Social media")])
# htmlcode += widgets.row([social.to_html()])
if not gh.empty:
    htmlcode += widgets.row([widgets.h(3, "Github activity over time")])
    htmlcode += widgets.row([gh.to_html()])
body += widgets.add_tab("Due Diligence", htmlcode)

# Technical Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Simple moving average")])
htmlcode += widgets.row([sma])
htmlcode += widgets.row([widgets.h(3, "Exponential moving average")])
htmlcode += widgets.row([ema])
htmlcode += widgets.row([widgets.h(3, "Zero lag exponential moving average")])
htmlcode += widgets.row([zlma])
htmlcode += widgets.row([widgets.h(3, "Commodity channel index")])
htmlcode += widgets.row([cci])
htmlcode += widgets.row([widgets.h(3, "Moving average convergence/divergence")])
htmlcode += widgets.row([macd])
htmlcode += widgets.row([widgets.h(3, "Fisher transform")])
htmlcode += widgets.row([fisher])
htmlcode += widgets.row([widgets.h(3, "Aroon indicator")])
htmlcode += widgets.row([aroon])
htmlcode += widgets.row([widgets.h(3, "Bollinger bands")])
htmlcode += widgets.row([bbands])
htmlcode += widgets.row([widgets.h(3, "Fibonacci retracement")])
htmlcode += widgets.row([fib])
body += widgets.add_tab("Technical Analysis", htmlcode)

# Quantitative Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Summary") + summary.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Normality statistics and tests") + normality.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Box and whisker plot")])
htmlcode += widgets.row([bw])
htmlcode += widgets.row([widgets.h(3, "Rolling mean and std deviation of prices")])
htmlcode += widgets.row([rolling])
htmlcode += widgets.row([widgets.h(3, "Rolling kurtosis of distribution of prices")])
htmlcode += widgets.row([kurtosis])
body += widgets.add_tab("Quantitative Analysis", htmlcode)

body += widgets.tab_clickable_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# # to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)